# 뉴스 데이터 전처리

이 노트북은 크롤링한 뉴스 데이터를 BERT 모델 학습을 위해 전처리하는 과정을 담고 있습니다.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import torch
from tqdm import tqdm

## 1. 데이터 로드 및 병합

In [2]:
# 데이터 경로 설정
data_path = Path('../data')

# 모든 CSV 파일 리스트 가져오기
csv_files = list(data_path.glob('*.csv'))

# 데이터프레임 리스트 생성
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    # 언론사 이름 추출 (파일명에서)
    press_name = file.stem.split('_')[0]
    df['press'] = press_name
    dfs.append(df)

# 모든 데이터프레임 병합
combined_df = pd.concat(dfs, ignore_index=True)
print(f"총 {len(combined_df)} 개의 기사가 있습니다.")

총 85698 개의 기사가 있습니다.


## 2. 텍스트 전처리

In [3]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    # 특수문자 제거
    text = re.sub(r'[^\w\s가-힣]', ' ', text)
    # 연속된 공백 제거
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# 결측치 제거
combined_df = combined_df.dropna(subset=['title', 'content'])

# 제목과 본문 전처리
combined_df['title_cleaned'] = combined_df['title'].apply(clean_text)
combined_df['content_cleaned'] = combined_df['content'].apply(clean_text)

# 제목과 본문 결합
combined_df['text'] = combined_df['title_cleaned'] + ' ' + combined_df['content_cleaned']

# 너무 짧은 기사 제거 (예: 100자 미만)
combined_df = combined_df[combined_df['text'].str.len() >= 100]

# 중복 기사 제거 (제목과 본문이 동일한 경우)
combined_df = combined_df.drop_duplicates(subset=['title_cleaned', 'content_cleaned'])

# 중복 제거 후 데이터 수 확인
print(f"중복 제거 후 남은 기사 수: {len(combined_df)}")

중복 제거 후 남은 기사 수: 79024


In [10]:
# 전처리 전
combined_df["title"].iloc[:5], combined_df["content"].iloc[:5]

(0             한 총리 "의료개혁특위, 의료계 · 전문가·환자 모두 포괄해 구성"
 1             민주 김준혁, '이대생 성상납 · 위안부' 발언 사과…"진심 반성"
 2    TV 토론 격돌한 명룡…이재명 "사탕발림 옳지 않다"…원희룡 "2년간 한 게 없다"
 3               민주당, '이대생 성 상납 발언' 김준혁에 "부적절…사과 권고"
 4            민주 김준혁, '이대생 성 상납 · 위안부' 발언 사과…"진심 반성"
 Name: title, dtype: object,
 0    한덕수 국무총리는 윤석열 대통령이 제안해 출범하게 될 대통령 직속 의료개혁특별위원회...
 1    더불어민주당 경기 수원정 김준혁 후보는 '이화여대 김활란 초대 총장이 미군에게 학생...
 2    4·10 총선 인천 계양을 지역구에서 이른바 '명룡 대전'을 벌이는 더불어민주당 이...
 3    더불어민주당은 오늘(2일) '이화여대 김활란 초대 총장이 미군에게 학생들을 성 상납...
 4    더불어민주당 김준혁(경기 수원정) 후보는 오늘(2일) '이화여대 김활란 초대 총장이...
 Name: content, dtype: object)

In [9]:
# 전처리 후
combined_df["title_cleaned"].iloc[:5], combined_df["content_cleaned"].iloc[:5]

(0              한 총리 의료개혁특위 의료계 전문가 환자 모두 포괄해 구성
 1                민주 김준혁 이대생 성상납 위안부 발언 사과 진심 반성
 2    TV 토론 격돌한 명룡 이재명 사탕발림 옳지 않다 원희룡 2년간 한 게 없다
 3                민주당 이대생 성 상납 발언 김준혁에 부적절 사과 권고
 4               민주 김준혁 이대생 성 상납 위안부 발언 사과 진심 반성
 Name: title_cleaned, dtype: object,
 0    한덕수 국무총리는 윤석열 대통령이 제안해 출범하게 될 대통령 직속 의료개혁특별위원회...
 1    더불어민주당 경기 수원정 김준혁 후보는 이화여대 김활란 초대 총장이 미군에게 학생들...
 2    4 10 총선 인천 계양을 지역구에서 이른바 명룡 대전 을 벌이는 더불어민주당 이재...
 3    더불어민주당은 오늘 2일 이화여대 김활란 초대 총장이 미군에게 학생들을 성 상납시켰...
 4    더불어민주당 김준혁 경기 수원정 후보는 오늘 2일 이화여대 김활란 초대 총장이 미군...
 Name: content_cleaned, dtype: object)

In [12]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79024 entries, 0 to 85697
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            79024 non-null  object
 1   content          79024 non-null  object
 2   press            79024 non-null  object
 3   url              79024 non-null  object
 4   created_date     79024 non-null  object
 5   modified_date    59642 non-null  object
 6   journalist       67600 non-null  object
 7   comment_count    79024 non-null  int64 
 8   related_titles   78522 non-null  object
 9   related_urls     78522 non-null  object
 10  title_cleaned    79024 non-null  object
 11  content_cleaned  79024 non-null  object
 12  text             79024 non-null  object
dtypes: int64(1), object(12)
memory usage: 8.4+ MB


## 3. 레이블링

언론사별로 편향성 레이블을 부여합니다.

In [ ]:
# 언론사별 편향성 레이블 매핑


## 4. 데이터셋 분할

In [ ]:
# train/validation/test 분할 (70/15/15)
train_df, temp_df = train_test_split(combined_df, test_size=0.3, random_state=42, stratify=combined_df['bias'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['bias'])

print(f"Train set: {len(train_df)} samples")
print(f"Validation set: {len(val_df)} samples")
print(f"Test set: {len(test_df)} samples")

## 5. BERT 토크나이징 준비

In [ ]:
# BERT 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

def prepare_bert_input(text, max_length=512):
    # 토크나이징
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return encoding

# 데이터셋 저장
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)
test_df.to_csv('test.csv', index=False)